In [40]:
import pandas as pd

from backtest.metric import Metric

In [47]:
factor = pd.read_pickle('factor_returns_0.pickle')
factor = factor.pct_change()
factor.index[0].year

2012

In [48]:
regime = pd.read_csv('./business_cycle.csv', index_col=0, parse_dates=True)
regime = regime.loc[str(factor.index[0].year):,]
regime.index = factor.index

In [49]:
fr_df = pd.concat([regime, factor], axis=1)
fr_df

,deflation,inflation,recovery,expansion,beta,mom,vol,prophet
2012-01-31,0,0,1,0,NaN,NaN,NaN,NaN
2012-02-29,0,0,1,0,0.038980,0.039295,0.008994,0.027363
2012-03-30,0,0,1,0,0.022468,0.028678,0.012645,0.016363
2012-04-30,0,0,1,0,-0.018273,0.022353,0.012155,-0.022182
2012-05-31,1,0,0,0,-0.067927,-0.026378,0.000600,-0.047736
...,...,...,...,...,...,...,...,...
2022-08-31,0,1,0,0,-0.038543,0.022801,-0.005979,-0.014942
2022-09-30,0,1,0,0,-0.060539,-0.049583,-0.043845,-0.041547
2022-10-31,0,1,0,0,0.019661,0.097215,0.039859,0.058380
2022-11-30,0,1,0,0,0.043822,0.002103,0.018022,0.023041


In [50]:
def check_best_regime(multi_asset_df, freq: str, _plot: bool=False):
    each_regime = multi_asset_df.columns[-4:]
    tickers = multi_asset_df.columns[:-4]
    
    _dict = {}
    for regime in each_regime:
        
        _temp = {}
        if _plot:
            print((1+multi_asset_df[tickers].mul(multi_asset_df[regime], axis=0)).cumprod().plot(lw=1))

        else:
            for ticker in tickers:
                _temp[ticker] = Metric(portfolio=(1 + multi_asset_df[ticker] * multi_asset_df[regime]).cumprod(), freq=freq).numeric_metric()
                print(f'{ticker} with {regime}')
                Metric(portfolio=(1 + multi_asset_df[ticker] * multi_asset_df[regime]).cumprod(), freq=freq).numeric_metric()
        _dict[regime] = _temp
    
    return _dict

In [51]:
test = check_best_regime(fr_df, freq='M', _plot=False)

deflation with beta
inflation with beta
recovery with beta
expansion with beta
deflation with mom
inflation with mom
recovery with mom
expansion with mom
deflation with vol
inflation with vol
recovery with vol
expansion with vol
deflation with prophet
inflation with prophet
recovery with prophet
expansion with prophet


In [52]:
test

{'beta': {'deflation': {'returns': '116.22',
   'CAGR': '1.38',
   'volatility': '0.10',
   'skewness': '1.96',
   'kurtosis': '19.48',
   'MDD': '-0.18',
   'MDD_duration': '1249.00',
   'sharp': '-0.26',
   'sortino': '-0.28',
   'calmar': '0.08',
   'VaR': '-0.08',
   'VaR_ratio': '0.02',
   'CVaR': '-0.10',
   'CVaR_ratio': '0.02',
   'hit': '0.59',
   'GtP': '0.96'},
  'inflation': {'returns': '92.36',
   'CAGR': '-0.72',
   'volatility': '0.05',
   'skewness': '-1.47',
   'kurtosis': '23.54',
   'MDD': '-0.11',
   'MDD_duration': '213.00',
   'sharp': '-0.99',
   'sortino': '-1.00',
   'calmar': '-0.06',
   'VaR': '-0.06',
   'VaR_ratio': '-0.01',
   'CVaR': '-0.07',
   'CVaR_ratio': '-0.01',
   'hit': '0.50',
   'GtP': '0.72'},
  'recovery': {'returns': '211.86',
   'CAGR': '7.06',
   'volatility': '0.07',
   'skewness': '4.76',
   'kurtosis': '31.00',
   'MDD': '-0.04',
   'MDD_duration': '729.00',
   'sharp': '0.43',
   'sortino': '3.09',
   'calmar': '1.96',
   'VaR': '-0.02'